## Explore the sizes and magntidues of objects

#### Standard imports

In [ ]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl

#### Change this to match your data location

In [ ]:
# pz_dir = '/global/cfs/cdirs/lsst/groups/PZ/DP1'
# if that fails you can use this
# pz_dir = '/global/u2/e/echarles/dx'
pz_dir = '/Users/echarles/pz'

In [ ]:
t = tables_io.read(f'{pz_dir}/data/dp1_v29.0.0/5063/object.parquet')

#### Set up the cosmology and a evaluation grid

In [ ]:
cosmo = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)
z_grid = np.linspace(0., 6., 301)[1:]
a_grid = (1/(1+z_grid))
D_a = cosmo.angular_diameter_distance(a_grid)
rad_to_asec = 360*60*60/(2*np.pi) 
ang_scale = D_a * 1000/ rad_to_asec
dist_mod = cosmo.distance_modulus(a_grid)

#### Some useful functions

In [ ]:
def r_95_of_n(n):
    return 1.25 + 1.125*n

def pix_to_radians(pix):
    return np.radians(0.2*pix/3600)

def mag_to_dL(mag, abs_mag=20):
    return np.power(10, 1+(mag+abs_mag)/5)

def pix_to_dA(pix, size=5e3):
    return size/pix_to_radians(pix)

#### Extract lots of interesting quantities

In [ ]:
blendedness = t['i_blendedness'].to_numpy()
blendedness = np.where(np.isnan(blendedness), 0., blendedness).clip(0., 1.)
mask  = np.bitwise_and(
    np.bitwise_and(
        t['i_extendedness'].to_numpy() == 1,
        t['g_psfFlux'].to_numpy() / t['g_psfFluxErr'].to_numpy() > 10,
    ),
    np.bitwise_and(
        t['i_psfFlux'].to_numpy() / t['i_psfFluxErr'].to_numpy() > 10,
        t['r_psfFlux'].to_numpy() / t['r_psfFluxErr'].to_numpy() > 10,
    ),
)
mask_unblend = np.bitwise_and(mask, blendedness<0.1)
mask_blend = np.bitwise_and(mask, blendedness>0.1)
blend = t['detect_fromBlend'].to_numpy()
psf_flux = t['i_psfFlux'].to_numpy()
cModel_flux = t['i_cModelFlux'].to_numpy()
cModel_flux = t['i_sersicFlux'].to_numpy()
shape_xx = t['i_ixx'].to_numpy()
shape_yy = t['i_iyy'].to_numpy()
shape_xy = t['i_ixy'].to_numpy()
psf_xx = t['i_ixxDebiasedPSF'].to_numpy()
psf_yy = t['i_iyyDebiasedPSF'].to_numpy()
psf_xy = t['i_ixyDebiasedPSF'].to_numpy()
shape_det = shape_xx*shape_yy - shape_xy*shape_xy
shape_trace = shape_xx + shape_yy
shape_area = np.sqrt(shape_det)
psf_det = psf_xx*psf_yy - psf_xy*psf_xy
psf_trace = psf_xx + psf_yy
sersic_x = t['sersic_reff_x'].to_numpy()
sersic_y = t['sersic_reff_y'].to_numpy()
sersic_rho = t['sersic_rho'].to_numpy()
sersic_index = t['sersic_index'].to_numpy()
sersic_factor = r_95_of_n(sersic_index)
sersic_xx = sersic_x*sersic_x
sersic_yy = sersic_y*sersic_y
sersic_trace = sersic_xx + sersic_yy
sersic_xy = sersic_x*sersic_y*sersic_rho
sersic_det = sersic_xx*sersic_yy - sersic_xy*sersic_xy
bdReB = t['i_bdReB'].to_numpy()
bdReD = t['i_bdReD'].to_numpy()
kron_rad = t['i_kronRad'].to_numpy()
psf_mag = -2.5 * np.log10(t['i_psfFlux'].to_numpy()) + 31.4
cModel_mag = -2.5 * np.log10(t['i_cModelFlux'].to_numpy()) + 31.4
sersic_mag = -2.5 * np.log10(t['i_sersicFlux'].to_numpy()) + 31.4

#### Angular sizes in radians

In [ ]:
pix_to_radians(np.sqrt(sersic_trace))

#### Equivaltent distance for an object with a 5kpc size

In [ ]:
5e3/pix_to_radians(np.sqrt(sersic_trace))

#### Magntidue to luminosity distance

In [ ]:
mag_to_dL(sersic_mag)

#### Plot estimated angular diameter distance (assuming 5kpc scale) v. lumi distance (assuing -20 abs mag)

In [ ]:
_ = plt.hist2d(mag_to_dL(sersic_mag)/1e9, pix_to_dA(np.sqrt(sersic_trace))/1e9, bins=(np.linspace(0., 20., 101), np.linspace(0, 20., 101)), norm='log')
_ = plt.xlabel(r'$d_{L}$ [Gpc]')
_ = plt.ylabel(r'$d_{A}$ [Gpc]')

#### Infer the redshift from the ratio of the estimated lumi distance to angular diameter distance

In [ ]:
_ = plt.hist(np.sqrt(mag_to_dL(sersic_mag)/pix_to_dA(np.sqrt(sersic_trace)))-1, bins=np.linspace(-1., 30., 101))
_ = plt.xlabel(r'Implied Redshit: $z_{\rm imp} = \sqrt{\frac{D_{L}}{D_{A}}}-1$')